In [ ]:
iwd = %pwd
%cd '/path to FEM auxiliary input scipt dir/'
pwd = %pwd

In [ ]:

import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(14)
tf.config.threading.set_intra_op_parallelism_threads(14)

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=13000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

import time

from pddlvm.utilities import tf, tfp, np, plt, tfb, floatf, tffloat, tfint, updateTfFloat,  _0
updateTfFloat ( tf.float32 )
tf.keras.backend.set_floatx('float32')

from pddlvm.poisson1DPDEState import PDE_State
from pddlvm.poisson1DSolver import Solver 
from pddlvm.NeuralNet import NeuralN
from VEModelBunny import VE
tf.random.set_seed(42)

import time

from scipy.sparse import csc_matrix

print(tffloat(0.))
tf.config.list_physical_devices()

prefix  = 'mirrorArch'
dirname = '{}/'.format(prefix)
os.makedirs(os.path.dirname(dirname), exist_ok=True)

In [ ]:
from scipy.sparse import csc_matrix
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

In [ ]:
import bunny

FEBunny = bunny.Bunny()

print('Istanciated')
FEBunny.initialize( 0 )
print('Initialized')

MeshNodes = tffloat(FEBunny.getNodeCoords())

coeffs = tf.reshape(tf.constant([ 1. ]), [-1,1])
FEBunny.setKappaCoeffs( coeffs )

A =  FEBunny.updateMaterialStiffnessAndReturn()

A = convert_sparse_matrix_to_sparse_tensor(A)
print(type(A))

_ = FEBunny.AssembleAndSolve()

ForcingVect = tffloat( FEBunny.getForcingVect( ) )
ForcingVect = tf.reshape( ForcingVect, [-1,1] )

normForcingVect = tf.linalg.norm(ForcingVect)
A = None

In [ ]:

maxi = tf.reduce_max(MeshNodes, axis = 0) + 10.
mini = tf.reduce_min(MeshNodes, axis = 0) - 10.
a   = tffloat([-1., -1., -1.]) 
b   = tffloat([ 1.,  1.,  1.]) 

def toByUnitCube(MeshNodes):
    return (b-a) * (MeshNodes - mini) / (maxi - mini) + a
    
def fromByUnitCube(cubeNodes):
    return (maxi - mini) * (cubeNodes - a) /(b - a) + mini

def fromByUnitYLine( col ):
    maxY = tf.reduce_max( MeshNodes[:,col] )
    minY = tf.reduce_min( MeshNodes[:,col] )
    a, b = -1., 1.
    return (b-a) * (MeshNodes[:,col] - minY) / (maxY - minY) + a

def chebyTrig(x, n):
    return tf.math.cos(n * tf.math.acos(x))

def VolumeChebyMesh(MeshCoords, numN):
    N = tf.range(0, numN, dtype=floatf)
    def bind(N): return chebyTrig(MeshCoords, N)
    T3DMesh = tf.vectorized_map(bind, N)
    return T3DMesh

numN = 7

uCubeNodes = toByUnitCube(MeshNodes)
T3DMesh = VolumeChebyMesh(uCubeNodes, numN)

def chebyToFE(uc):
    return tf.einsum('nmli,nk,mk,lk->ki', uc, T3DMesh[:,:,0] , T3DMesh[:,:,1], T3DMesh[:,:,2])

def chebyToFE_diagCov(uc):
    return tf.einsum('nk,mk,lk,nmli,nk,mk,lk->ki',
                      T3DMesh[:,:,0] , T3DMesh[:,:,1], T3DMesh[:,:,2],
                      uc,
                      T3DMesh[:,:,0] , T3DMesh[:,:,1], T3DMesh[:,:,2]
                     )


In [ ]:
proposalU = tf.random.uniform(minval=0., maxval=0.1, shape=[numN,numN,numN,3], dtype=floatf)
print(proposalU)
dispField = chebyToFE(proposalU)
print(dispField)
# raise
print(uCubeNodes[:,0])
print(dispField)
#%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
angle = 0
rot   = 0

fig = plt.figure()
ax = Axes3D(fig)

img = ax.scatter(uCubeNodes[:,0], uCubeNodes[:,2], uCubeNodes[:,1], c=dispField[:,0], cmap=plt.hot(), alpha=0.5)
plt.colorbar( img )
ax.view_init(angle, rot)
plt.show()
plt.close()

fig = plt.figure()
ax = Axes3D(fig)
img = ax.scatter(uCubeNodes[:,0], uCubeNodes[:,2], uCubeNodes[:,1], c=dispField[:,1], cmap=plt.hot(), alpha=0.5)
plt.colorbar( img )
ax.view_init(angle, rot)
plt.show()
plt.close()


fig = plt.figure()
ax = Axes3D(fig)
img = ax.scatter(uCubeNodes[:,0], uCubeNodes[:,2], uCubeNodes[:,1], c=dispField[:,2], cmap=plt.hot(), alpha=0.5)
plt.colorbar( img )
ax.view_init(angle, rot)
plt.show()
plt.close()


In [ ]:
import types

dimK  = 3
model = VE( numN, dimK )

def setKappaFunc(self, kappa):
    '''kappa must be nx1'''
    kappa = tf.reshape(kappa, [-1])
    tf.py_function(FEBunny.setKappaCoeffs, [kappa], [])

model.setKappaFunc = types.MethodType(setKappaFunc, model)



def createNNs():
    nh   = 2_500
    numU = tf.reduce_prod( model.shapeU )
    numK = tf.reduce_prod( model.shapeK )
    U_K  = NeuralN('U_of_K', 
                    np.array([  numK,
                                nh, nh, nh,
                                numU])
                    )

    K_U  = NeuralN('K_of_U',
                    np.array([  numU, 
                                nh, nh, nh,
                                numK ]),
                        )

    model.initPriorNetworks(U_K=U_K, K_U=K_U)
    model.setModePrior()
    return 

model.LatentKappaPrior   = tfp.distributions.Uniform(low=tf.constant(-2., shape=model.shapeK), high=tf.constant(2., shape=model.shapeK))

createNNs()


In [ ]:

def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

def getFromBunnyAndConvert( ):
    Asps = FEBunny.updateMaterialStiffnessAndReturn( )
    Asps = convert_sparse_matrix_to_sparse_tensor( Asps )
    return [Asps.indices, tf.cast(Asps.values, floatf), Asps.dense_shape]


Indeces, _ , DenseShape = getFromBunnyAndConvert( )
model.setKappaFunc( model.LatentKappaPrior.mean( ) )
PreCTriplet = getFromBunnyAndConvert( )
SparsePre   = tf.SparseTensor(PreCTriplet[0], tf.cast(PreCTriplet[1], tf.float32), PreCTriplet[2]) 
PreCTriplet = None
DensePreInv = tf.sparse.to_dense( SparsePre ) 
DensePreInv = tf.linalg.inv( tf.sparse.to_dense( SparsePre )  )
DensePre = None
PreCTriplet = None
SparsePre = None


In [ ]:
@tf.custom_gradient
def returnAtsSparse( ):
    ATriple = tf.py_function(getFromBunnyAndConvert, [], [tf.int64, floatf, tf.int64])
    def grad( dy ):
        return dy
    return ATriple[1], grad

def logProbPhysResidual(self, uc, epsilon):
    uc   = tf.reshape(uc, self.shapeU)
    u    = tf.reshape(chebyToFE( uc ), [-1,1] )
    AValues  = returnAtsSparse( ) 
    A    =   tf.SparseTensor(Indeces, AValues, DenseShape) 
    r    = DensePreInv @ ( tf.sparse.sparse_dense_matmul(A,  u) -  ForcingVect )
    logP = - 0.5 * ( u.shape[0] * tf.math.log(epsilon**2) 
                        + tf.squeeze( tf.linalg.norm(r)**2/epsilon**2 ) + self.log2pi )
    return logP
model.logProbPhysResidual = types.MethodType(logProbPhysResidual, model)


In [ ]:
def NNSolve( kappaSample ):

    mu_alpha, logvar_alpha = model.priorModels['U_of_K'].Map( [tf.reshape(kappaSample, [1,-1])] ) 
    uc   = tf.reshape(mu_alpha, model.shapeU)
    u    = chebyToFE(uc)
    solutionField = tf.reshape(u,[-1,1])
    
    return solutionField, None

from scipy import sparse

def solveFEModel( kappaSample ):
    model.setKappaFunc( kappaSample )
    FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1,1]) )
    Acsc = FEBunny.updateMaterialStiffnessAndReturn( )
    u    = sparse.linalg.spsolve(Acsc, ForcingVect)
    return u

In [ ]:
def compareNNAndFE(numTimes):
    for i in range(numTimes):
        kappaSample = model.LatentKappaPrior.sample(sample_shape=())
        print(kappaSample)
        model.setKappaFunc( kappaSample )
        FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1]) )
        dispField = solveFEModel( kappaSample )
        dispField = tf.reshape(dispField, [-1,3])
        fig = plt.figure()
        ax  = Axes3D(fig)
        img = ax.scatter(uCubeNodes[:,0], uCubeNodes[:,2], uCubeNodes[:,1], c=dispField[:,1], cmap=plt.hot(), alpha=0.5)
        plt.colorbar( img )
        ax.view_init(angle, rot)
        plt.title('FEModel Solve')
        plt.show()
        plt.close()

        dispField, StdBunny = NNSolve( kappaSample )
        dispField = tf.reshape(dispField, [-1,3])
        print("dispField", dispField)
        fig = plt.figure()
        ax = Axes3D(fig)
        img = ax.scatter(uCubeNodes[:,0], uCubeNodes[:,2], uCubeNodes[:,1], c=dispField[:,1], cmap=plt.hot(), alpha=0.5)
        plt.colorbar( img )
        ax.view_init(angle, rot)
        plt.title('NN Solve')
        plt.show()


compareNNAndFE(2)

In [ ]:
eps =1e-2

AllEpsU = []
AllEpsK = []
AllEpsB = []

totalTimeTraining  = 0.

trainModels        = False

intervalELBOsave   = 100

PLOT = True


@tf.function
def train_step_prior(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = model.compute_nelbo_prior(iteration)
    for priorvars in model.priorModels['trainable_variables']:
        tape.watch( priorvars )
        gradients    = tape.gradient(loss, priorvars)
        optimizer.apply_gradients(zip(gradients, priorvars))
    return loss

if trainModels:
       
    '''Optimization sequence'''
    model.n_sgd      = 1

    num_iterations   = tfint(50_000)

    model.epsilon_r.assign( tffloat(eps) )

    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-3, int(num_iterations/5), 0.5, staircase=True, name=None
    )
    optimizer = tf.keras.optimizers.Adam( lr, clipnorm= 5. )

    elboAll       = []
    startTraining = time.time()
    for epoch in tf.range(0, num_iterations ):

        start_time = time.time()
        loss = train_step_prior(optimizer, epoch)

        end_time = time.time()

        if epoch % intervalELBOsave == 0:

            elboAll.append(-loss)

            np.savetxt( '{}elboAll.dat'.format(dirname), np.array(elboAll) )
            with open('{}times.dat'.format(dirname), 'a') as file:
                file.write('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}\n'
                    .format(epoch, -loss, end_time - start_time))

    endTraining = time.time()
    totalTimeTraining += endTraining - startTraining
    print("DONE TRAINING FOR EPSI", eps)




In [ ]:
'''If you want to save or load Model Weights'''

saveModelWeights = True
loadModelWeights = False



if saveModelWeights:
    model.priorModels['U_of_K'].NN.save_weights(dirname+'U_of_K_weights.h5')
    model.priorModels['K_of_U'].NN.save_weights(dirname+'K_of_U_weights.h5')
    model.priorModels['U_of_K'].NN.save(dirname+'U_of_K_weights.model')
    model.priorModels['K_of_U'].NN.save(dirname+'K_of_U_weights.model')
    print('SAVED MODEL WEIGHTS')
    

if loadModelWeights:

    model.priorModels['U_of_K'].NN = tf.keras.models.load_model(dirname+'U_of_K_weights.model')
    model.priorModels['K_of_U'].NN = tf.keras.models.load_model(dirname+'K_of_U_weights.model')

    model.priorModels['U_of_K'].NN.compile()
    model.priorModels['K_of_U'].NN.compile()
    # Check its architecture
    model.priorModels['U_of_K'].NN.summary()
    model.priorModels['K_of_U'].NN.summary()
    print('LOADED MODEL WEIGHTS')


In [ ]:
plt.close()

elboAll = np.array(elboAll)
scale = 1e8
fig, axs = plt.subplots()
axs.plot(range(elboAll.shape[0]),  elboAll )
axs.set_ylim(-scale,scale)
plt.show()
print(elboAll)

In [ ]:
TimeFETot = 0
TimeNNFWTot = 0
TimeNNInvTot = 0

TotNSEFW    = 0
TotNSEINV   = 0

normSolutionFE = np.array([])

from pddlvm.poisson1DPDEState import PDE_State
yLine = fromByUnitYLine( 1 )
print('yLine = ', yLine)
pdeState = PDE_State(yLine, dimK=3)

transform         = tfb.Chain( bijectors=[tfb.Shift(tffloat(1.)), tfb.Softplus()] )
pdeState.kappaBij = transform


numIndptSampels = 100
for i in range( numIndptSampels ):
    
    kappaSample = model.LatentKappaPrior.sample(sample_shape=())
    print(kappaSample)
    model.setKappaFunc( kappaSample )
    
    #True Solving
    startTimeFE = time.time()
    FEBunny.updateMaterialStiffnessAndReturn()
    X_ = tf.cast(FEBunny.AssembleAndSolve( ), tf.float32 )
    normSolutionFE = np.append(normSolutionFE, np.array([tf.linalg.norm(X_)]))
    iterTimeFE  = time.time() - startTimeFE
    print('iterTimeFE = ', iterTimeFE)
    TimeFETot  += iterTimeFE
    
    #NN Forward
    startTimeNN = time.time()
    mu_alpha, logvar_alpha = model.priorModels['U_of_K'].Map( [tf.reshape(kappaSample, [1,-1])] ) 
    uc     = tf.reshape(mu_alpha, model.shapeU)
    uNN    = tf.reshape( chebyToFE(uc), [-1] )
    iterTimeNNFW  = time.time() - startTimeNN
    print('iterTimeNNFW = ', iterTimeNNFW)
    TimeNNFWTot  += iterTimeNNFW
    
    
    #NN Inv
    startTimeNN = time.time()
    mu_beta, logvar_beta = model.priorModels['K_of_U'].Map( [tf.reshape(uc, [1,-1])] ) 
    
    mean, diagCov = pdeState.gaussChebyGaussCoeffs( tf.squeeze(mu_beta), tf.squeeze(tf.exp(logvar_beta)), x=yLine )
    ybar, yvar    = pdeState.unscentedGaussTransf(mean, diagCov, transform=transform, k=1)

    iterTimeNNINV  = time.time() - startTimeNN
    print('iterTimeNNINV = ', iterTimeNNINV)
    TimeNNInvTot  += iterTimeNNINV
    pdeState.kappa = tf.squeeze( kappaSample )
    kappaTrueOnElm = pdeState.kappa_fun(yLine)
    
    NSEINV     = tf.linalg.norm(  kappaTrueOnElm - ybar )**2 / tf.linalg.norm( kappaTrueOnElm )**2
    print("NSEINV = ", NSEINV)
    TotNSEINV += NSEINV
    print("TotNSEINV = ", TotNSEINV)
    
    NSEFW     = tf.linalg.norm(  X_ - uNN )**2 / tf.linalg.norm( X_ )**2
    print("NSEFW = ", NSEFW)
    TotNSEFW += NSEFW
    print("TotNSEFW = ", TotNSEFW)
    
    
AvgNSEFW  = TotNSEFW / numIndptSampels
AvgNSEINV = TotNSEINV / numIndptSampels

print("AvgNSEFW = ", AvgNSEFW)
print("AvgNSEINV = ", AvgNSEINV)

print("TimeFETot = ", TimeFETot)
print("TimeNNFWTot = ",  TimeNNFWTot)
print("TimeNNInvTot = ", TimeNNInvTot)

TimeFEAVG = TimeFETot / numIndptSampels
TimeNNFWAVG = TimeNNFWTot / numIndptSampels
TimeNNInvAVG = TimeNNInvTot / numIndptSampels

print("TimeFEAVG = ", TimeFEAVG)
print("TimeNNFWAVG = ",  TimeNNFWAVG)
print("TimeNNInvAVG = ", TimeNNInvAVG)

avgNormSlnFE = tf.reduce_mean(normSolutionFE)
stddevNormSlnFE = tf.math.reduce_std(normSolutionFE)
print('avgNormSlnFE = ', avgNormSlnFE)
print('stddevNormSlnFE = ', stddevNormSlnFE)


In [ ]:
dirnameFile = dirname+'test4'

FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )

kappaSample = model.LatentKappaPrior.sample(sample_shape=())
print(kappaSample)
kappaSample = tf.constant([[0.739],[ 0.167 ],[ -1.136 ]], dtype=tf.float32)

model.setKappaFunc( kappaSample )
FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1]) )
A =  FEBunny.updateMaterialStiffnessAndReturn()

mu_alpha, logvar_alpha = model.priorModels['U_of_K'].Map( [tf.reshape(kappaSample, [1,-1])] ) 
uc   = tf.reshape(mu_alpha, model.shapeU)
u    = chebyToFE(uc)

solutionField = u
FEBunny.setSolution(tf.reshape(tf.cast( solutionField , tf.float64), [-1]) )
FEBunny.writeVTUFile('{}solutionFieldMean.vtu'.format(dirnameFile))

FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )
model.setKappaFunc( kappaSample )
FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1]) )

lgVarc   = tf.reshape(logvar_alpha, model.shapeU)
Var      = tf.exp(lgVarc)
VarBunny = chebyToFE_diagCov( Var )
StddevBunny = tf.sqrt( tf.reshape(VarBunny, [-1]) )
A =  FEBunny.updateMaterialStiffnessAndReturn()
FEBunny.setSolution( tf.reshape(tf.cast(StddevBunny, tf.float64) , [-1] ) )
FEBunny.writeVTUFile('{}solutionFieldStddev.vtu'.format(dirnameFile))


FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )
FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1,1]) )
A =  FEBunny.updateMaterialStiffnessAndReturn()
X_ = FEBunny.AssembleAndSolve( )
FEBunny.writeVTUFile('{}trueSolution.vtu'.format(dirnameFile))

FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )
FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1,1]) )
A =  FEBunny.updateMaterialStiffnessAndReturn()
FEBunny.setSolution( tf.abs(X_ - tf.reshape(tf.cast( solutionField , tf.float64), [-1]) )  ) 
FEBunny.writeVTUFile('{}absErrorDisp.vtu'.format(dirnameFile))

FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )
FEBunny.setSolution(tf.reshape(tf.cast( solutionField , tf.float64), [-1]) )
mu_beta, logvar_beta = model.priorModels['K_of_U'].Map( [tf.reshape(mu_alpha, [1,-1])] ) 
FEBunny.setKappaCoeffs( tf.reshape(mu_beta,[-1]) )
A =  FEBunny.updateMaterialStiffnessAndReturn()
FEBunny.setSolution(tf.reshape(tf.cast( solutionField , tf.float64), [-1]) )
FEBunny.writeVTUFile('{}kappaMeanReconstructed.vtu'.format(dirnameFile))


In [ ]:
from pddlvm.poisson1DPDEState import PDE_State
yLine = fromByUnitYLine( 1 )
print('yLine = ', yLine)
pdeState = PDE_State(yLine, dimK=3)

transform     = tfb.Chain( bijectors=[tfb.Shift(tffloat(1.)), tfb.Softplus()] )
mean, diagCov = pdeState.gaussChebyGaussCoeffs( tf.squeeze(mu_beta), tf.squeeze(tf.exp(logvar_beta)), x=yLine )
ybar, yvar    = pdeState.unscentedGaussTransf(mean, diagCov, transform=transform, k=1)

YBAR          = tf.reshape(tf.tile(ybar[:,None], [1, 3]), [-1])
YSTD          = tf.sqrt( tf.reshape(tf.tile(yvar[:,None], [1, 3]), [-1]) )
print('YBAR = ', YBAR)
print('YSTD = ', YSTD)

FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )
FEBunny.setKappaCoeffs( tf.reshape(kappaSample,[-1]) )
A =  FEBunny.updateMaterialStiffnessAndReturn()
FEBunny.setSolution(tf.reshape(tf.cast( YBAR , tf.float64), [-1]) )
FEBunny.writeVTUFile('{}kappaMeanReconstructed--unscented.vtu'.format(dirnameFile))

FEBunny = bunny.Bunny()
FEBunny.initialize( 0 )
FEBunny.setSolution(tf.reshape(tf.cast( YSTD , tf.float64), [-1]) )
FEBunny.writeVTUFile('{}kappaVarReconstructed--unscented.vtu'.format(dirnameFile))
